In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df_clean = pd.read_csv('SmartCleaned.csv')
model_browser = df_clean[['auction_id', 'experiment', 'date', 'hour', 'device_make', 'browser', 'brand_awareness']].copy()

In [5]:
#Splitting into train set, test set, and validation set
train_browser, test_browser, validate_browser  = \
              np.split(model_browser[model_browser.columns[1:]].sample(frac=1, random_state=42), 
                       [int(.7*len(model_browser)), int(.9*len(model_browser))])

In [6]:
browser_x = model_browser.columns[1:-1]

browser_train =  train_browser[browser_x]
browser_train_y = train_browser['brand_awareness']

browser_test =  test_browser[browser_x]
browser_test_y = test_browser['brand_awareness']

browser_validate =  validate_browser[browser_x]
browser_validate_y = validate_browser['brand_awareness']

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [8]:
numerical_browser = ['date','hour']
cat_browser = ['experiment', 'device_make', 'browser']

In [9]:
class ExtractDay(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        pass
    def fit(self, X, y=None):
        return self               # take the date column and pss to self
    
    def transform(self, X, y=None):
        X = X.copy()
        X['date'] = X['date'].apply(lambda x:x.weekday())
        return X

In [10]:
num_pipeline = Pipeline([
    ('day_extractor', ExtractDay()),
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, numerical_browser),
    ('cat', OneHotEncoder(), numerical_browser)
])

In [11]:
processed_train_browser = full_pipeline.fit_transform(browser_train)
processed_test_browser = full_pipeline.fit_transform(browser_test)
processed_validate_browser = full_pipeline.fit_transform(browser_validate)

AttributeError: 'str' object has no attribute 'weekday'

In [12]:
df_clean

,Unnamed: 0,auction_id,experiment,date,hour,device_make,platform_os,browser,brand_awareness
0,0,d3b96730-3fd5-4f03-9432-4ef4c1160cac,exposed,2020-07-04,7,Generic Smartphone,6,Chrome Mobile,0
1,1,f27fb7fa-fe23-43d4-bcd0-8baa18774dd3,control,2020-07-03,15,Pixel 2,6,Facebook,1
2,2,1831572b-470a-47ae-957c-ddff6a8e8126,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
3,3,f74343a5-f659-4062-b237-2e4f636339cc,control,2020-07-04,7,Samsung SM-G930F,6,Samsung Internet,0
4,4,e3499355-c235-47af-ad5e-e98c06227867,control,2020-07-05,5,Generic Smartphone,6,Chrome Mobile,0
...,...,...,...,...,...,...,...,...,...
1238,1238,36355037-f07b-4ee1-af6f-441dc51b9f6e,exposed,2020-07-07,11,Generic Smartphone,6,Chrome Mobile,0
1239,1239,fa8ed69f-6b89-47d2-8ece-c5e1f4973207,control,2020-07-04,2,Samsung SM-G930F,6,Facebook,0
1240,1240,6bbc74ed-546f-48b5-bf91-cb8888e96286,exposed,2020-07-05,9,Samsung SM-J530F,6,Samsung Internet,1
1241,1241,a3942624-335c-4567-9f98-78ccbb5791d0,exposed,2020-07-03,5,XiaoMi MI MAX 3,6,Chrome Mobile WebView,1
